<a href="https://colab.research.google.com/github/dattali18/IR_Assignments/blob/main/Assignment.05/IR_05_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas nltk

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
# download the files from github

pro_israel_url = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.05/data/Word/pro-israel.txt?raw=true"
pro_palestine_url = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.05/data/Word/pro-palestine.txt?raw=true"
anti_israel_url = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.05/data/Word/anti-israel.txt?raw=true"
anti_palestine_url = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.05/data/Word/anti-palestine.txt?raw=true"

sentences_url = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.05/data/sentences/sentences.csv?raw=true"

In [ ]:
# download all the file into the same dir as the note book

import urllib.request

# Download files
urllib.request.urlretrieve(pro_israel_url, 'pro-israel.txt')
urllib.request.urlretrieve(pro_palestine_url, 'pro-palestine.txt')
urllib.request.urlretrieve(anti_israel_url, 'anti-israel.txt')
urllib.request.urlretrieve(anti_palestine_url, 'anti-palestine.txt')
urllib.request.urlretrieve(sentences_url, 'sentences.csv')

print("Files downloaded successfully!")

In [ ]:
import pandas as pd
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# Initialize the stemmer
stemmer = PorterStemmer()

# Function to load and stem words from a file
def load_and_stem_words(file_path):
    with open(file_path, 'r') as file:
        words = file.read().splitlines()
    stemmed_words = {stemmer.stem(word) for word in words}
    return stemmed_words

# Load and stem words for each class
pro_israel_words = load_and_stem_words('pro-israel.txt')
anti_palestine_words = load_and_stem_words('anti-palestine.txt')
pro_palestine_words = load_and_stem_words('pro-palestine.txt')
anti_israel_words = load_and_stem_words('anti-israel.txt')

# Load sentences from the CSV file
sentences_df = pd.read_csv('sentences.csv')

# Function to classify a sentence
def classify_sentence(sentence):
    # Tokenize and stem the sentence
    tokens = word_tokenize(sentence.lower())
    stemmed_tokens = {stemmer.stem(token) for token in tokens}
    
    # Check which class the sentence belongs to
    if stemmed_tokens & pro_israel_words and not (stemmed_tokens & anti_palestine_words | stemmed_tokens & pro_palestine_words | stemmed_tokens & anti_israel_words):
        return 'pro-israel'
    elif stemmed_tokens & anti_palestine_words and not (stemmed_tokens & pro_israel_words | stemmed_tokens & pro_palestine_words | stemmed_tokens & anti_israel_words):
        return 'anti-palestine'
    elif stemmed_tokens & pro_palestine_words and not (stemmed_tokens & pro_israel_words | stemmed_tokens & anti_palestine_words | stemmed_tokens & anti_israel_words):
        return 'pro-palestine'
    elif stemmed_tokens & anti_israel_words and not (stemmed_tokens & pro_israel_words | stemmed_tokens & anti_palestine_words | stemmed_tokens & pro_palestine_words):
        return 'anti-israel'
    else:
        return 'neutral'

# Apply classification to each sentence
sentences_df['classification'] = sentences_df['sentence'].apply(classify_sentence)

# Save classified sentences to separate files
for classification in ['pro-israel', 'anti-palestine', 'pro-palestine', 'anti-israel', 'neutral']:
    classified_sentences = sentences_df[sentences_df['classification'] == classification]
    classified_sentences.to_csv(f'{classification}_classified.csv', index=False)

print("Classification complete! Check the generated CSV files.")